In [367]:
import random

In [366]:
def weights_dp(cap: int, p: list, w: list):
    '''DP on weights'''
    import time
    start = time.time()
    n = len(w)
    dp = [[0 for i in range(cap + 1)] for i in range(n + 1)]
    items = []
    for i in range(1, n + 1):
        for j in range(1, cap + 1):
            wi = w[i - 1]
            if wi <= j:
                dp[i][j] = max(dp[i - 1][j], p[i - 1] + dp[i - 1][j - wi])
            else:
                dp[i][j] = dp[i - 1][j]
                
    def findAns(dp: list, i: int, j: int, w: list):
        if dp[i][j] == 0: 
            return
        if dp[i - 1][j] == dp[i][j]:
            findAns(dp, i - 1, j, w)
        else:
            findAns(dp, i - 1, j - w[i - 1], w)
            items.append(i - 1)
            
    findAns(dp, n, cap, w)        
    indexes = []
    for i in range(n):
        if i in items:
            indexes.append(1)
        else:
            indexes.append(0)
    
    end = time.time()
    return indexes, dp[n][cap], end - start


def count_profit(items: list, p: list):
    profit = 0
    for i, item in enumerate(items):
        profit += item * p[i]
    
    return profit


def count_weights(items: list, w: list):
    weight = 0
    for i, item in enumerate(items):
        weight += item * w[i]
    
    return weight


def upper_bound(arr: list, target: int):
    for i, item in enumerate(arr):
        if item > target:
            return i

In [2]:
def correct_chromosome(chromosome: list, weights: list, profits: list, max_weight: int):
    '''Corrects chromosome id it does not fit the capacity condition'''
    from random import randint
    one_indices = [i for i, item in enumerate(chromosome) if item == 1]
    cur_weight = count_weights(chromosome, weights)
    cur_profit = count_profit(chromosome, profits)
    while cur_weight > max_weight:
        cur_index = one_indices[randint(0, len(one_indices) - 1)]
        one_indices.remove(cur_index)
        chromosome[cur_index] = 0
        cur_weight -= weights[cur_index]
        cur_profit -= profits[cur_index]
    return chromosome, cur_profit, cur_weight

In [3]:
def generate_chromosome(capacity: int, profits: list, weights: list, ):
    '''Generates one chromosome'''
    from random import random
    chromosome = []
    for i in range(len(weights)):
        gene = 0 if random() < 0.5 else 1
        chromosome.append(gene)
    chromosome, profit, weight = correct_chromosome(chromosome, weights, profits, capacity)
    return chromosome, profit, weight

In [5]:
def mutate_chromosome(chromosome: list):
    '''Toggles one random bit of a chromosome'''
    mut_index = random.randint(0, len(chromosome) - 1)
    chromosome[mut_index] = int(not chromosome[mut_index])
    return chromosome

In [331]:
N_GROUPS = 4
SELECT_INTERVALS = [0.1, 0.25, 0.5, 1] #4 группы, вероятности 10%, 20%, 30%, 40%

def genetic_alg(cap: int, profits: list, weights: list, mutation_prob = 0.15, n_chromosomes = 8, n_iterations=30):
    '''Genetic algorithm for a knapsack task'''
    import time
    start = time.time()
    group_size = n_chromosomes / N_GROUPS
    chromosomes_list = []
    for i in range(n_chromosomes):
        chromosomes_list.append(generate_chromosome(cap, profits, weights))
    chromosomes_list = sorted(chromosomes_list, key=lambda item: item[1])
    
    best_chromosome = chromosomes_list[-1]
    
    for i in range(n_iterations):
        group1_index = upper_bound(SELECT_INTERVALS, random.random()) + 1
        chromo1_index = random.randint((group1_index - 1) * group_size, group1_index * group_size - 1)
        
        group2_index = upper_bound(SELECT_INTERVALS, random.random()) + 1
        chromo2_index = random.randint((group1_index - 1) * group_size, group1_index * group_size - 1)
        
        while chromo2_index == chromo1_index:
            group2_index = upper_bound(SELECT_INTERVALS, random.random()) + 1
            chromo2_index = random.randint((group1_index - 1) * group_size, group1_index * group_size - 1)
            
        chromo1 = chromosomes_list[chromo1_index][0]
        chromo2 = chromosomes_list[chromo2_index][0]
        
        swap_index = random.randint(0, len(chromo1) - 1)
        part1_1, part2_1= chromo1[:swap_index], chromo2[:swap_index]
        
        chromo1 = part2_1 + chromo1[swap_index:]
        chromo2 = part1_1 + chromo2[swap_index:]
        
        if random.random() < mutation_prob:
            if random.random() < 0.5:
                chromo1 = mutate_chromosome(chromo1)
            else:
                chromo2 = mutate_chromosome(chromo2)
        
        chromo1 = correct_chromosome(chromo1, weights, profits, cap)
        chromo2 = correct_chromosome(chromo2, weights, profits, cap)
        
        # print(chromo1, chromo2)
        
        chromosomes_list[chromo1_index] = chromo1
        chromosomes_list[chromo2_index] = chromo2
        
        chromosomes_list = sorted(chromosomes_list, key=lambda item: item[1])
        
        # print(chromosomes_list[-1])
        
        if chromosomes_list[-1][1] > best_chromosome[1]:
            best_chromosome = chromosomes_list[-1]
     
    end = time.time()
    return best_chromosome, end - start
    
    

In [7]:
folder = '../Lab2/benchmarks/'
files_list = []
for i in range(1, 8):
    files_list.append((f'{folder}p0{i}_c.txt', f'{folder}p0{i}_w.txt', f'{folder}p0{i}_p.txt', f'{folder}p0{i}_s.txt'))

In [364]:
max_profits = []
found_profits = []
dp_times = []
genetic_alg_times = []
for i, sample in enumerate(files_list):
    with open(sample[0], 'r') as c, open(sample[1], 'r') as w, open(sample[2], 'r') as p, open(sample[3], 'r') as s:
        capacity = int(c.read())
        weights = list(map(int, w.read().split()))
        profits = list(map(int, p.read().split()))
        answer = list(map(int, s.read().split()))
        dp_items, dp_ans, dp_time = weights_dp(cap=capacity, p=profits, w=weights)
        dp_times.append(dp_time)
        max_profits.append(dp_ans)
        assert answer == dp_items
        print(f'capacity: {capacity}')
        print(f'max profit: {dp_ans}')
        print(f'optimal items: {answer}')
        print(f'dp algorithm time: {dp_time}')
        (alg_items, alg_profit, alg_weight), alg_time = genetic_alg(capacity, profits, 
                                                                    weights, n_chromosomes=16, n_iterations=50)
        genetic_alg_times.append(alg_time)
        found_profits.append(alg_profit)
        print('----------------------')
        print(f'GENETIC ALGORITHM RESULTS:')
        print(f'items: {alg_items}')
        print(f'profit: {alg_profit}')
        print(f'weight: {alg_weight}')
        print(f'time: {alg_time}')
        print('----------------------\n')
        

capacity: 165
max profit: 309
optimal items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
dp algorithm time: 0.0009999275207519531
----------------------
GENETIC ALGORITHM RESULTS:
items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
profit: 309
weight: 165
time: 0.0010018348693847656
----------------------

capacity: 26
max profit: 51
optimal items: [0, 1, 1, 1, 0]
dp algorithm time: 0.0
----------------------
GENETIC ALGORITHM RESULTS:
items: [0, 1, 1, 1, 0]
profit: 51
weight: 26
time: 0.000499725341796875
----------------------

capacity: 190
max profit: 150
optimal items: [1, 1, 0, 0, 1, 0]
dp algorithm time: 0.0004990100860595703
----------------------
GENETIC ALGORITHM RESULTS:
items: [1, 1, 0, 0, 1, 0]
profit: 150
weight: 190
time: 0.0009989738464355469
----------------------

capacity: 50
max profit: 107
optimal items: [1, 0, 0, 1, 0, 0, 0]
dp algorithm time: 0.0
----------------------
GENETIC ALGORITHM RESULTS:
items: [1, 0, 0, 1, 0, 0, 0]
profit: 107
weight: 50
time: 0.000997304916381836
----------------

In [365]:
import pandas as pd
stats = pd.DataFrame()
stats['max profit'] = max_profits
stats['max profit by genetic alg'] = found_profits
stats['dp alg time'] = dp_times
stats['genetic alg time'] = genetic_alg_times
stats

,max profit,max profit by genetic alg,dp alg time,genetic alg time
0,309,309,0.001000,0.001002
1,51,51,0.000000,0.000500
2,150,150,0.000499,0.000999
3,107,107,0.000000,0.000997
4,900,883,0.000499,0.000499
5,1735,1735,0.000501,0.000500
6,1458,1449,0.004497,0.001001
